In [118]:
import pandas as pd
import numpy as np
import csv
import datetime as dt
from datetime import timedelta
from datetime import datetime
import time
import matplotlib as plt
import statistics
import math

In [119]:
data_start = '2019-4-15'
data_end = '2019-4-30'

res = (datetime.strptime(data_end, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
adr = pd.read_csv('analytics_data_report')
df = pd.DataFrame(adr)
df.columns = df.columns.str.replace(' ', '_').str.lower()
t1 = pd.to_datetime(df['referral_received'])
t2 = pd.to_datetime(df['loan_booked'])
mask = (t1 <= res) & (t1 >= data_start)
conv = df.loc[mask]
bookcount = conv.groupby('partner')['loan_booked'].count()
leadcount = conv.groupby('partner')['referral_received'].count()
conversions = bookcount / leadcount
conv['difference'] = t2 - t1
conv['difference'] = conv['difference'] / dt.timedelta(days=1)
df_dates = conv[['partner', 'referral_received', 'loan_booked', 'difference']]
mean_diff = df_dates.groupby('partner')['difference'].mean()
med_diff = df_dates.groupby('partner')['difference'].median()
df_summ = pd.DataFrame({'Volume': leadcount,
                       'Booked/App': conversions,
                       'Mean Time to Fund': mean_diff,
                       'Median Time to Fund': med_diff})
result = pd.merge(df, df_summ, on='partner')
result = result[np.isfinite(result['Median Time to Fund'])]
result = result[np.isfinite(result['Mean Time to Fund'])]
start = pd.to_datetime(result['referral_received'])
result['p_fund_median'] = start + result['Median Time to Fund'].map(dt.timedelta)
result['p_fund_mean'] = start + result['Mean Time to Fund'].map(dt.timedelta)
df_summ.dropna()

C:\Users\rjanssen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\rjanssen\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,Volume,Booked/App,Mean Time to Fund,Median Time to Fund
partner,,,,
AVIS,94,0.372340,16.891290,16.236111
CarGurus - Avis,10,0.300000,15.718287,14.188194
Chrysler Capital Lease,79,0.316456,18.851639,19.615972
Credit Karma,681,0.080764,23.770278,22.403472
Donlen,17,0.647059,15.219823,13.629167
Element,36,0.222222,24.168576,20.921875
Emkay,7,0.714286,23.409583,17.468056
Enterprise Fleet Management,77,0.766234,19.986323,17.904861
Fiserv GM Lease,74,0.351351,23.053632,21.586806


In [120]:
start_date = '2019-6-1'
end_date = '2019-6-13'

res2 = (datetime.strptime(end_date, '%Y-%m-%d') + timedelta(days=1)).strftime('%Y-%m-%d')
pred_median = (result['p_fund_median'] <= res2) & (result['p_fund_median'] >= start_date)
new_median = result.loc[pred_median]
appcount_median = new_median.groupby('partner')['referral_received'].count()
predicted_median = appcount_median * conversions
Total_median = predicted_median.sum()
pred_mean = (result['p_fund_mean'] < res2) & (result['p_fund_mean'] >= start_date)
new_mean = result.loc[pred_mean]
appcount_mean = new_mean.groupby('partner')['referral_received'].count()
predicted_mean = appcount_mean * conversions
Total_mean = predicted_mean.sum()
booked = pd.to_datetime(result['loan_booked'])
actual = (booked <= res2) & (booked >= start_date)
actual2 = result.loc[actual]
true_actual = actual2.groupby('partner')['loan_booked'].count()
prediction_test = pd.DataFrame({'Predicted Median': round(predicted_median,0),
                           'Predicted Mean': round(predicted_mean,0),
                           'Actual': true_actual})
prediction_test.loc['Total'] = prediction_test.sum()
prediction_test.dropna()

,Predicted Median,Predicted Mean,Actual
AVIS,29.0,31.0,33.0
Chrysler Capital Lease,22.0,22.0,17.0
Credit Karma,42.0,42.0,41.0
Donlen,3.0,4.0,3.0
Element,5.0,5.0,10.0
Emkay,1.0,1.0,3.0
Enterprise Fleet Management,51.0,54.0,65.0
Fiserv GM Lease,30.0,30.0,29.0
HCA - Hyundai Kia,20.0,18.0,14.0
IFS,10.0,9.0,20.0


In [148]:
result['test']= result['p_fund_median'].dt.strftime('%m/%d/%Y')